In [2]:
import os
import pandas as pd
import time
from openai import OpenAI
from dotenv import load_dotenv

# 🔑 API 키 로드
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# 🔍 감성 + 프레임 분류 함수
def classify_sentiment_and_frame(comment):
    try:
        prompt = f"""다음 유튜브 댓글을 감성과 프레임 관점에서 분석해주세요.

댓글: "{comment}"

1. 감성: 긍정 / 부정 / 중립 중에서 하나로 분류
2. 프레임: 지지 / 비판 / 조롱 / 정보 / 기타 중에서 하나로 분류

응답 형식:
감성: (선택)
프레임: (선택)
"""

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )

        result = response.choices[0].message.content.strip().split('\n')
        sentiment = result[0].replace('감성:', '').strip()
        frame = result[1].replace('프레임:', '').strip()
        return sentiment, frame

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return "분석 실패", "분석 실패"

# 📄 분석 파일
input_path = "youtube_통합분석_김문수_후보_등록.csv"
output_path = "김문수_감성프레임분석_샘플.csv"

# 🔽 데이터 로딩 및 샘플링
df = pd.read_csv(input_path)
df_sample = df.head(100).copy()

# 🔁 분석 수행
sentiments, frames = [], []
for i, comment in enumerate(df_sample['comment']):
    sentiment, frame = classify_sentiment_and_frame(comment)
    sentiments.append(sentiment)
    frames.append(frame)
    print(f"{i+1:>3} ▶ 감성: {sentiment} | 프레임: {frame}")
    time.sleep(1.2)

# ➕ 결과 저장
df_sample['sentiment'] = sentiments
df_sample['frame'] = frames
df_sample.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n✅ 분석 결과 저장 완료 → {output_path}")

# 📊 통계 요약 생성
sentiment_stats = df_sample['sentiment'].value_counts().to_frame(name='count')
sentiment_stats['비율(%)'] = round((sentiment_stats['count'] / len(df_sample)) * 100, 1)

frame_stats = df_sample['frame'].value_counts().to_frame(name='count')
frame_stats['비율(%)'] = round((frame_stats['count'] / len(df_sample)) * 100, 1)

pivot_table = pd.crosstab(df_sample['sentiment'], df_sample['frame'], normalize='index') * 100
pivot_table = pivot_table.round(1)

# ✅ 콘솔 미리보기
print("\n📊 감성 분포:\n", sentiment_stats)
print("\n📊 프레임 분포:\n", frame_stats)
print("\n📊 감성 × 프레임 조합 (%):\n", pivot_table)

# ✅ CSV 저장
sentiment_stats.to_csv("김문수_감성_분석_요약.csv", encoding='utf-8-sig')
frame_stats.to_csv("김문수_프레임_분석_요약.csv", encoding='utf-8-sig')
pivot_table.to_csv("_김문수_감성x프레임_매트릭스.csv", encoding='utf-8-sig')
print("\n📁 통계 요약 저장 완료")


  1 ▶ 감성: 긍정 | 프레임: 지지
  2 ▶ 감성: 긍정 | 프레임: 지지
  3 ▶ 감성: 부정 | 프레임: 비판
  4 ▶ 감성: 부정 | 프레임: 비판
  5 ▶ 감성: 부정 | 프레임: 비판
  6 ▶ 감성: 부정 | 프레임: 비판
  7 ▶ 감성: 부정 | 프레임: 비판
  8 ▶ 감성: 부정 | 프레임: 정보
  9 ▶ 감성: 부정 | 프레임: 지지
 10 ▶ 감성: 부정 | 프레임: 비판
 11 ▶ 감성: 긍정 | 프레임: 지지
 12 ▶ 감성: 긍정 | 프레임: 지지
 13 ▶ 감성: 부정 | 프레임: 비판
 14 ▶ 감성: 부정 | 프레임: 비판
 15 ▶ 감성: 부정 | 프레임: 비판
 16 ▶ 감성: 긍정 | 프레임: 지지
 17 ▶ 감성: 긍정 | 프레임: 조롱
 18 ▶ 감성: 중립 | 프레임: 지지
 19 ▶ 감성: 부정 | 프레임: 비판
 20 ▶ 감성: 부정 | 프레임: 비판
 21 ▶ 감성: 부정 | 프레임: 지지
 22 ▶ 감성: 중립 | 프레임: 정보
 23 ▶ 감성: 긍정 | 프레임: 조롱
 24 ▶ 감성: 부정 | 프레임: 비판
 25 ▶ 감성: 부정 | 프레임: 비평
 26 ▶ 감성: 긍정 | 프레임: 지지
 27 ▶ 감성: 부정 | 프레임: 비판
 28 ▶ 감성: 부정 | 프레임: 비판
 29 ▶ 감성: 부정 | 프레임: 비판
 30 ▶ 감성: 긍정 | 프레임: 지지
 31 ▶ 감성: 부정 | 프레임: 비판
 32 ▶ 감성: 부정 | 프레임: 비판
 33 ▶ 감성: 긍정 | 프레임: 지지
 34 ▶ 감성: 부정 | 프레임: 비판
 35 ▶ 감성: 부정 | 프레임: 비판
 36 ▶ 감성: 부정 | 프레임: 비판
 37 ▶ 감성: 긍정 | 프레임: 조롱
 38 ▶ 감성: 부정 | 프레임: 비판
 39 ▶ 감성: 부정 | 프레임: 비판
 40 ▶ 감성: 부정 | 프레임: 비판
 41 ▶ 감성: 긍정 | 프레임: 기다림/기대
 42 ▶ 감성: 부정 | 프레임: 비판
 43 ▶ 감성: 부정 | 프레임: 비판
 44 ▶ 감